<a href="https://colab.research.google.com/github/Achiever-caleb/Kaggle-Facial-recognition-Project/blob/main/Copy_of_facial_keypoint_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
import zipfile

In [ ]:
data_path = "/content/facial-keypoints-detection.zip"
extract_path = "/tmp/facial-keypoints-detection" # Use a temporary directory

# Create the extraction directory if it doesn't exist
import os
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the main data file
with zipfile.ZipFile(data_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Unzip the training data
with zipfile.ZipFile(f"{extract_path}/training.zip", "r") as zip_ref:
    zip_ref.extractall(f"{extract_path}/training_data")

# Unzip the test data
with zipfile.ZipFile(f"{extract_path}/test.zip", "r") as zip_ref:
    zip_ref.extractall(f"{extract_path}/test_data")

# Read the training and test data into Pandas DataFrames
train = pd.read_csv(f'{extract_path}/training_data/training.csv')
test = pd.read_csv(f'{extract_path}/test_data/test.csv')
id = pd.read_csv(f"{extract_path}/IdLookupTable.csv")

In [ ]:
display(train.head())

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [ ]:
display(test.head())

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [ ]:
display(id.head())

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN


In [ ]:
train.isnull().sum()

,0
left_eye_center_x,10
left_eye_center_y,10
right_eye_center_x,13
right_eye_center_y,13
left_eye_inner_corner_x,4778
left_eye_inner_corner_y,4778
left_eye_outer_corner_x,4782
left_eye_outer_corner_y,4782
right_eye_inner_corner_x,4781
right_eye_inner_corner_y,4781


In [ ]:
train.fillna(method='ffill', inplace=True)
train.isnull().sum()

/tmp/ipython-input-2125910151.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method='ffill', inplace=True)


,0
left_eye_center_x,0
left_eye_center_y,0
right_eye_center_x,0
right_eye_center_y,0
left_eye_inner_corner_x,0
left_eye_inner_corner_y,0
left_eye_outer_corner_x,0
left_eye_outer_corner_y,0
right_eye_inner_corner_x,0
right_eye_inner_corner_y,0


In [ ]:
m,n = train.shape
print(m, '\t', n)

7049 	 31


In [ ]:
test.shape

(1783, 2)

In [ ]:
img = []
img_size = 96
for i in range(m):
    spliting = np.array(train['Image'][i].split(' '),dtype = 'float64')
    splitting = np.reshape(spliting,(img_size,img_size,1))
    splitting /= 255
    img.append(splitting)
img = np.array(img)

In [ ]:
X_train = img
train.drop('Image', axis = 1, inplace = True)
y_train = []

for i in range(len(train)):
    y = train.iloc[i,:].values
    y_train.append(y)


y_train= np.array(y_train, dtype = 'float')

In [ ]:
model = Sequential([
    Conv2D(128, (3, 3),strides=1,activation='relu',padding = 'same', input_shape=(96, 96, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), strides=1,activation='relu',padding = 'same'),
    Dropout(0.1),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), strides=1,activation='relu',padding = 'same'),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), strides=1,activation='relu',padding = 'same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.1),
    Dense(256, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dense(30)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mae',metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train.reshape(-1, 96, 96, 1), y_train, epochs=2, batch_size=32,validation_split=0.2)

Epoch 1/2
177/177 ━━━━━━━━━━━━━━━━━━━━ 731s 4s/step - accuracy: 0.3018 - loss: 9.7204 - val_accuracy: 0.6809 - val_loss: 9.6854
Epoch 2/2
177/177 ━━━━━━━━━━━━━━━━━━━━ 702s 4s/step - accuracy: 0.5031 - loss: 4.0971 - val_accuracy: 0.6844 - val_loss: 6.1378


In [ ]:
X_test = np.array([np.fromstring(x, dtype=int, sep=' ') for x in test['Image']])
X_test = X_test.reshape(-1, 96, 96, 1) / 255.0

y_test_pred = model.predict(X_test)

56/56 ━━━━━━━━━━━━━━━━━━━━ 42s 754ms/step


In [ ]:
y_test_pred

array([[56.0771  , 30.84226 , 25.0523  , ..., 63.206238, 40.67748 ,
        66.20054 ],
       [54.993565, 30.31039 , 24.581139, ..., 61.961723, 40.290688,
        64.79552 ],
       [59.465393, 32.880577, 26.665194, ..., 67.16079 , 43.57455 ,
        69.838684],
       ...,
       [59.929092, 32.955296, 26.4071  , ..., 68.30336 , 43.420425,
        70.50163 ],
       [59.3352  , 32.605972, 25.9648  , ..., 67.48821 , 42.818035,
        70.15665 ],
       [58.832653, 32.56329 , 26.556015, ..., 66.01142 , 42.92433 ,
        69.81919 ]], dtype=float32)

In [ ]:
test_images = []
for i in range(len(test)):
    item = np.array(test['Image'][i].split(' '), dtype='float')
    item = np.reshape(item, (img_size, img_size, 1))
    item /= 255
    test_images.append(item)

test_images = np.array(test_images, dtype='float')

In [ ]:
predict = model.predict(test_images)

56/56 ━━━━━━━━━━━━━━━━━━━━ 42s 745ms/step


In [ ]:
feature_names = list(id['FeatureName'])
image_ids = list(id['ImageId']-1)
row_ids = list(id['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))

In [ ]:
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(predict[x][y])


In [ ]:
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('s.csv',index = False)

##Version 2 Submission

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,      # Rotate images by up to 20 degrees
    width_shift_range=0.1,  # Shift images horizontally by up to 10%
    height_shift_range=0.1, # Shift images vertically by up to 10%
    shear_range=0.1,        # Apply shearing transformations
    zoom_range=0.1,         # Zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Fill missing pixels with the nearest value
)

# Fit the data generator on the training data
datagen.fit(X_train)

In [ ]:
 data_path = "/content/facial-keypoints-detection.zip"
extract_path = "/tmp/facial-keypoints-detection" # Use a temporary directory

# Create the extraction directory if it doesn't exist
import os
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the main data file
import zipfile
with zipfile.ZipFile(data_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Unzip the training data
with zipfile.ZipFile(f"{extract_path}/training.zip", "r") as zip_ref:
    zip_ref.extractall(f"{extract_path}/training_data")

# Unzip the test data
with zipfile.ZipFile(f"{extract_path}/test.zip", "r") as zip_ref:
    zip_ref.extractall(f"{extract_path}/test_data")

# Read the training data into Pandas DataFrames
train = pd.read_csv(f'{extract_path}/training_data/training.csv')

print("Missing values before imputation:")
print(train.isnull().sum())

# Apply median imputation, excluding the 'Image' column
keypoint_columns = train.columns.drop('Image')
train[keypoint_columns] = train[keypoint_columns].fillna(train[keypoint_columns].median())


print("\nMissing values after median imputation:")
print(train.isnull().sum())

# Prepare image data
img_size = 96
X_train = []
for i in range(len(train)):
    spliting = np.array(train['Image'][i].split(' '), dtype='float64')
    splitting = np.reshape(spliting, (img_size, img_size, 1))
    splitting /= 255
    X_train.append(splitting)
X_train = np.array(X_train)

# Prepare target variables
y_train = train.drop('Image', axis=1).values

Missing values before imputation:
left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_b

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create data generators for training and validation
datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_val = ImageDataGenerator() # No augmentation for validation data

# Fit the training data generator
datagen_train.fit(X_train_split)


# Train the model using the data generators
history_improved = model_improved.fit(datagen_train.flow(X_train_split, y_train_split, batch_size=32),
                                      epochs=5, # You can adjust the number of epochs
                                      validation_data=datagen_val.flow(X_val_split, y_val_split, batch_size=32))

Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


177/177 ━━━━━━━━━━━━━━━━━━━━ 136s 767ms/step - accuracy: 0.0490 - loss: 35.5820 - val_accuracy: 0.0021 - val_loss: 9.1813
Epoch 2/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 134s 757ms/step - accuracy: 0.3748 - loss: 5.6054 - val_accuracy: 0.5298 - val_loss: 2.7736
Epoch 3/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 140s 792ms/step - accuracy: 0.5228 - loss: 4.2942 - val_accuracy: 0.5475 - val_loss: 2.0078
Epoch 4/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 137s 773ms/step - accuracy: 0.5302 - loss: 4.0123 - val_accuracy: 0.5957 - val_loss: 1.8695
Epoch 5/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 134s 759ms/step - accuracy: 0.5236 - loss: 3.8155 - val_accuracy: 0.5738 - val_loss: 1.8325


In [ ]:
# Re-load test and id data
extract_path = "/tmp/facial-keypoints-detection"
test = pd.read_csv(f'{extract_path}/test_data/test.csv')
id = pd.read_csv(f"{extract_path}/IdLookupTable.csv")

# Prepare test image data
X_test = []
img_size = 96
for i in range(len(test)):
    item = np.array(test['Image'][i].split(' '), dtype='float')
    item = np.reshape(item, (img_size, img_size, 1))
    item /= 255
    X_test.append(item)

X_test = np.array(X_test, dtype='float')

# Generate predictions on the test data
predictions = model_improved.predict(X_test)

# Ensure predictions are within the valid range (0-96)
predictions = np.clip(predictions, 0.0, 96.0)

56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 176ms/step


In [ ]:
# Prepare submission file
submission_result = pd.DataFrame(columns=['RowId', 'Location'])

for i in range(len(id)):
    row_id = id['RowId'][i]
    image_id = id['ImageId'][i] - 1  # Adjust for 0-based indexing
    feature_name = id['FeatureName'][i]

    # Find the index of the feature name in the training columns
    # Ensure 'train' dataframe is available from previous steps
    if 'train' in locals() or 'train' in globals():
        feature_index = train.columns.get_loc(feature_name)
    else:
        # If 'train' is not available, reload it (assuming it's needed to get column order)
        extract_path = "/tmp/facial-keypoints-detection"
        train = pd.read_csv(f'{extract_path}/training_data/training.csv')
        # Apply median imputation as done before, excluding the 'Image' column
        keypoint_columns = train.columns.drop('Image')
        train[keypoint_columns] = train[keypoint_columns].fillna(train[keypoint_columns].median())
        feature_index = train.columns.get_loc(feature_name)


    location = predictions[image_id][feature_index]

    submission_result = pd.concat([submission_result, pd.DataFrame({'RowId': [row_id], 'Location': [location]})], ignore_index=True)

# Save the submission file
submission_result.to_csv('submission_improved.csv', index=False)

print("Submission file 'submission_improved.csv' created successfully.")

/tmp/ipython-input-598638552.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submission_result = pd.concat([submission_result, pd.DataFrame({'RowId': [row_id], 'Location': [location]})], ignore_index=True)


Submission file 'submission_improved.csv' created successfully.


In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)
model_improved.compile(optimizer=optimizer, loss='mae', metrics=['accuracy'])

model_improved.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 92, 92, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 46, 46, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 22, 22, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 10, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │        15,390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,505,374 (9.56 MB)

 Trainable params: 2,503,390 (9.55 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model_improved = Sequential([
    Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(y_train.shape[1])  # Output layer with 30 neurons for the keypoints
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
